In [1]:
import os, cv2, matplotlib as plt, numpy as np, pandas as pd,csv,glob
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import array
import tensorflow as tf
import copy
from tensorflow import keras

from tensorflow.keras import layers
import PIL
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from IPython.display import Image
from tensorflow.keras.callbacks import TensorBoard
import multiprocess
from setup_img_3_1 import *
import timeit

In [2]:
#reading the csv file of CASME II
df1 = pd.read_excel(r'D:\Masters project\Casme2\CASME2-coding-20190701.xlsx',converters={'Subject': lambda x: str(x),'Filename': lambda x: str(x)},header=None)

In [3]:
#extracting the pathfiles for Dense Optical Flow images
temppath="D:\Masters project\DenseSplit\\"
a=np.empty(len(df1.index)-1,dtype = 'object')  #store into numpy arrays

for i in range(len(df1.index)-1):
    a[i]=(temppath+'sub'+df1.loc[i+1,0]+'\\'+df1.loc[i+1,1])
    a[i]=(a[i].replace("\\", "/")) 
    
    
len(a) #

255

In [4]:
#Dropping the imbalanced expresseion sequences
df=df1.drop(0)
#print(df[0].unique())

subjects=df[0].unique()

df.head()
#print(subjects)
df.drop(df[df[8]=='others'].index, inplace = True)
df.drop(df[df[8]=='sadness'].index, inplace = True)
df.drop(df[df[8]=='fear'].index, inplace = True)
#df.drop(df[df[8]=='disgust'].index, inplace = True)
#df.drop(df[df[8]=='happiness'].index, inplace = True)

In [57]:
#max no of frames
maxframes=0
framecount=[]
for j in range(len(df.index)):
    result= df.iloc[j,4]-df.iloc[j,3]
    #print(result)
    framecount.append(result+3)
    
framecount
    
    

[16,
 23,
 38,
 48,
 51,
 68,
 68,
 31,
 33,
 56,
 33,
 36,
 21,
 63,
 18,
 58,
 23,
 32,
 34,
 40,
 33,
 43,
 36,
 31,
 46,
 3,
 28,
 33,
 35,
 26,
 23,
 33,
 58,
 48,
 36,
 31,
 40,
 56,
 41,
 81,
 68,
 48,
 58,
 41,
 46,
 38,
 38,
 28,
 63,
 46,
 48,
 71,
 23,
 72,
 23,
 43,
 38,
 41,
 26,
 46,
 48,
 56,
 58,
 28,
 26,
 53,
 51,
 36,
 28,
 36,
 43,
 24,
 91,
 8,
 61,
 18,
 31,
 31,
 23,
 46,
 24,
 39,
 69,
 38,
 51,
 36,
 26,
 21,
 23,
 12,
 41,
 53,
 48,
 20,
 25,
 48,
 28,
 31,
 21,
 26,
 43,
 25,
 24,
 36,
 71,
 28,
 31,
 31,
 46,
 23,
 36,
 26,
 21,
 26,
 33,
 28,
 21,
 33,
 72,
 68,
 53,
 43,
 78,
 38,
 38,
 41,
 46,
 23,
 39,
 26,
 20,
 23,
 25,
 26,
 28,
 31,
 33,
 23,
 43,
 36,
 28,
 33,
 25,
 42,
 31,
 28,
 46,
 41,
 24,
 46]

In [58]:
#df.head

In [59]:
df.iloc[0,4],df.iloc[0,3]

(59, 46)

In [60]:
#Extracting the Subjects for dataset
subjects, counts = np.unique(df[0].values,return_counts=True)

In [61]:
#subject path for the dataset using subjects
subpath=np.empty(len(subjects),dtype = 'object')
k=0
for i in range(len(subjects)):
    subpath[i]=(temppath+'sub'+subjects[k]+'\\')
    #subpath[i]=(subpath[i]+df1.loc[i+1,1])
    subpath[i]=(subpath[i].replace("\\", "/"))
    k=k+1
        
subpath[1]

'D:/Masters project/DenseSplit/sub02/'

In [62]:
#extrainct the emotion path based on Subject path
emotionp=[]*len(df[1])
for i in range(len(df[1])):
    emotionp.append(temppath+'sub'+df.iloc[i,0]+'\\'+df.iloc[i,1]+'/')
    emotionp[i]=(emotionp[i].replace("\\", "/"))
emotionp[i]

'D:/Masters project/DenseSplit/sub26/EP18_51/'

In [78]:
#empty list for pathfile of each emotion
pathd=[]

In [79]:
#appending all the images for the emotion paths
for i in range(len(emotionp)):
    pathff=[]
    img_file=framecount[i]-2
    files=glob.glob(emotionp[i]+str(img_file)+'.png')
    #last=max(files,key=os.path.getctime)    
    #print(last)
    for file in files:
        pathff.append(file)
        file=file.replace("\\", "/")
    #print(file)
    #print('')
    #print(file)
    #print('')
    ad=len(pathff)
    #print(ad)
    #PADDING
    #for j in range(maxframes-ad):
    #pathff.append(file)
    pathd.extend(pathff)

pathd[0]

'D:/Masters project/DenseSplit/sub01/EP02_01f/14.png'

In [80]:
#Correcting the Pathfiles
for i in range(len(pathd)):
        pathd[i]=pathd[i].replace("\\","/")

pathd[0]

'D:/Masters project/DenseSplit/sub01/EP02_01f/14.png'

In [81]:
pathd


['D:/Masters project/DenseSplit/sub01/EP02_01f/14.png',
 'D:/Masters project/DenseSplit/sub01/EP19_05f/21.png',
 'D:/Masters project/DenseSplit/sub01/EP19_06f/36.png',
 'D:/Masters project/DenseSplit/sub02/EP01_11f/46.png',
 'D:/Masters project/DenseSplit/sub02/EP02_04f/49.png',
 'D:/Masters project/DenseSplit/sub02/EP03_02f/66.png',
 'D:/Masters project/DenseSplit/sub02/EP06_01f/66.png',
 'D:/Masters project/DenseSplit/sub02/EP06_02f/29.png',
 'D:/Masters project/DenseSplit/sub02/EP09_01/31.png',
 'D:/Masters project/DenseSplit/sub02/EP11_01/54.png',
 'D:/Masters project/DenseSplit/sub02/EP13_04/31.png',
 'D:/Masters project/DenseSplit/sub02/EP14_01/34.png',
 'D:/Masters project/DenseSplit/sub03/EP01_2/19.png',
 'D:/Masters project/DenseSplit/sub03/EP07_04/61.png',
 'D:/Masters project/DenseSplit/sub03/EP09_03/16.png',
 'D:/Masters project/DenseSplit/sub03/EP18_06/56.png',
 'D:/Masters project/DenseSplit/sub03/EP19_08/21.png',
 'D:/Masters project/DenseSplit/sub04/EP12_01f/30.png',
 '

In [82]:
#pathd=np.reshape(pathd,(255,-1))
pathd=np.array(pathd)
pathd.shape

(148,)

In [83]:
#pathd[0]

In [84]:
#extracting the label paths
label=[]
for l in range(len(df[8])):
    label.append(df.iloc[l,8])
#df[8].unique()

In [85]:
len(label)

150

In [19]:
subjects

array(['01', '02', '03', '04', '05', '06', '07', '08', '09', '11', '12',
       '13', '14', '15', '16', '17', '19', '20', '21', '22', '23', '24',
       '25', '26'], dtype=object)

In [20]:
for i in subjects:
    print(i)

01
02
03
04
05
06
07
08
09
11
12
13
14
15
16
17
19
20
21
22
23
24
25
26


In [21]:
#One-hot encoding of labels
l=np.array(label)

le = LabelEncoder()
le.fit(l)
l

array(['happiness', 'disgust', 'disgust', 'repression', 'repression',
       'repression', 'repression', 'repression', 'happiness', 'surprise',
       'surprise', 'surprise', 'surprise', 'disgust', 'disgust',
       'disgust', 'disgust', 'disgust', 'disgust', 'disgust', 'surprise',
       'happiness', 'surprise', 'surprise', 'surprise', 'disgust',
       'surprise', 'happiness', 'surprise', 'surprise', 'disgust',
       'disgust', 'disgust', 'disgust', 'disgust', 'disgust',
       'repression', 'happiness', 'happiness', 'repression', 'happiness',
       'repression', 'repression', 'happiness', 'repression', 'happiness',
       'repression', 'disgust', 'disgust', 'disgust', 'disgust',
       'disgust', 'surprise', 'happiness', 'disgust', 'surprise',
       'surprise', 'surprise', 'happiness', 'disgust', 'disgust',
       'disgust', 'happiness', 'happiness', 'happiness', 'happiness',
       'happiness', 'happiness', 'surprise', 'disgust', 'happiness',
       'repression', 'happiness', 'h

In [22]:
integer_encoded= le.transform(l)
integer_encoded

array([1, 0, 0, 2, 2, 2, 2, 2, 1, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 3, 1,
       3, 3, 3, 0, 3, 1, 3, 3, 0, 0, 0, 0, 0, 0, 2, 1, 1, 2, 1, 2, 2, 1,
       2, 1, 2, 0, 0, 0, 0, 0, 3, 1, 0, 3, 3, 3, 1, 0, 0, 0, 1, 1, 1, 1,
       1, 1, 3, 0, 1, 2, 1, 1, 3, 1, 0, 0, 0, 2, 0, 1, 1, 1, 2, 2, 2, 2,
       0, 1, 2, 0, 0, 2, 0, 0, 2, 0, 0, 0, 1, 2, 0, 0, 1, 1, 3, 3, 1, 3,
       0, 0, 0, 3, 3, 0, 0, 2, 2, 2, 1, 0, 2, 2, 0, 0, 0, 2, 0, 0, 3, 3,
       3, 3, 0, 0, 3, 3, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0])

In [23]:
le.classes_

array(['disgust', 'happiness', 'repression', 'surprise'], dtype='<U10')

In [24]:
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
#integer_encoded1 = integer_encoded1.reshape(len(integer_encoded1), 1)
onehot_encoder = OneHotEncoder(sparse=False)

onehot_encoder.fit(integer_encoded)

label= onehot_encoder.transform(integer_encoded)
label

array([[0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],


In [25]:
#Leave one subject out training and testing

allTesting = []
allTraining=[]
Test_ground=[]
Train_ground=[]

for sindex in subjects: # range(len(subjects)):
    curSubject = "sub" + sindex
    print("TEST SUBJECT:", curSubject)
    #start = timeit.timeit()
    y_train=[]
    y_test=[]
    X_train=[]
    X_test=[]

    for lt in range(len(label)):
        if curSubject in pathd[lt]:
            y_test.append(label[lt])
            X_test.append(pathd[lt])
        else:
            y_train.append(label[lt])
            X_train.append(pathd[lt])
        
    X_train=np.array(X_train)
    X_test=np.array(X_test)
    y_train=np.array(y_train)
    y_test=np.array(y_test)
    print(len(X_train))
    print(len(X_test))
    with multiprocess.Manager() as manager:
        final_results = manager.dict()
        process_eval = multiprocess.Process(target=runTraining, args=(X_train, X_test, y_train, y_test,final_results))
        process_eval.start()
        process_eval.join()
        allTesting.extend(final_results.get("test"))
        allTraining.extend(final_results.get("train"))
        Train_ground.extend(final_results.get("train_ground"))
        Test_ground.extend(final_results.get("test_ground")) 
    #end = timeit.timeit()
    #print(end - start)
    #test1=final_results["test"]
    #allTesting.append(test1)
    





TEST SUBJECT: sub01


IndexError: index 148 is out of bounds for axis 0 with size 148

In [ ]:
allTesting

In [ ]:
allTesting = np.array(allTesting)
allTraining=np.array(allTraining)
Train_ground=np.array(Train_ground)
Test_ground=np.array(Test_ground)

In [ ]:
num_classes=3
allTesting=np.reshape(allTesting,(-1,))
allTraining=np.reshape(allTraining,(-1,))
Train_ground=np.reshape(Train_ground,(-1,))
Test_ground=np.reshape(Test_ground,(-1,))

In [ ]:
allTraining.shape

In [ ]:
    allTesting.shape

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix
import seaborn as sns

In [ ]:
#confusion Matrix
test_cmat = confusion_matrix(Test_ground, allTesting,labels=list(range(len(le.classes_))))
print(test_cmat)

In [ ]:
train_cmat = confusion_matrix(Train_ground, allTraining,labels=list(range(len(le.classes_))))
print(train_cmat)

In [ ]:
sns.heatmap(test_cmat, cmap="Blues", annot=True)

In [ ]:
sns.heatmap(train_cmat, cmap="Blues",annot=True)

In [ ]:
accuracy_score(Test_ground, allTesting)

In [ ]:
accuracy_score(Train_ground, allTraining)

In [ ]:
df_cm = pd.DataFrame(test_cmat, columns=le.classes_, index = list(range(len(le.classes_))))

In [ ]:
plt.figure(figsize = (10,7))
sns.set(font_scale=1.4)#for label size
sns.heatmap(df_cm, cmap="Blues", annot=True,annot_kws={"size": 16})

In [ ]:
df_cm1 = pd.DataFrame(train_cmat, columns=le.classes_, index = list(range(len(le.classes_))))

In [ ]:
plt.figure(figsize = (10,7))
sns.set(font_scale=1.4)#for label size
sns.heatmap(df_cm1, cmap="Blues", annot=True,annot_kws={"size": 16})